# NBA Win-Loss Modeling Data Prep



This Jupyter Notebook script will walk you through the process of engineering features using Team, Player, and Game data for the purpose of creating an NBA Home Team Win-Loss Binary Classification Model. Follow the link for inuition around the model https://medium.com/@theresearchlab/create-an-nba-win-loss-model-w-68-precision-d1c6a21f0ded

In [4]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests

implement a retry wrapper using a python decorator function. This will prevent the HTTP Timeouts from stopping the script's run during API calls. 

In [5]:
# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

In [6]:
# Get Season Schedule Function 

def getSeasonScheduleFrame(seasons, seasonType): 

    # Get date from string
    def getGameDate(matchup):
        return matchup.partition(' at')[0][:10]

    # Get Home team from string
    def getHomeTeam(matchup):
        return matchup.partition(' at')[2]

    # Get Away team from string
    def getAwayTeam(matchup):
        return matchup.partition(' at')[0][10:]

    # Match nickname from schedule to team table to find ID
    def getTeamIDFromNickname(nickname):
        return teamLookup.loc[teamLookup['nickname'] == difflib.get_close_matches(nickname, teamLookup['nickname'], 1)[0]].values[0][0] 
    
    @retry
    def getRegularSeasonSchedule(season, teamID, seasonType):
        season = str(season) + "-" + str(season+1)[-2:] # Convert year to season format ie. 2020 -> 2020-21
        teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00', season=season, season_type_all_star=seasonType, team_id=teamID).get_normalized_json()

        teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])
        teamGames['SEASON'] = season
        return teamGames    
    
    # Get team lookup table
    teamLookup = pd.DataFrame(teams.get_teams())
    
    # Get teams schedule for each team for each season
    scheduleFrame = pd.DataFrame()

    for season in seasons:
        for id in teamLookup['id']:
            time.sleep(1)
            scheduleFrame = pd.concat([scheduleFrame, getRegularSeasonSchedule(season, id, seasonType)], ignore_index=True)
            
    scheduleFrame['GAME_DATE'] = pd.to_datetime(scheduleFrame['MATCHUP'].map(getGameDate))
    scheduleFrame['HOME_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getHomeTeam)
    scheduleFrame['HOME_TEAM_ID'] = scheduleFrame['HOME_TEAM_NICKNAME'].map(getTeamIDFromNickname)
    scheduleFrame['AWAY_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getAwayTeam)
    scheduleFrame['AWAY_TEAM_ID'] = scheduleFrame['AWAY_TEAM_NICKNAME'].map(getTeamIDFromNickname)
    scheduleFrame = scheduleFrame.drop_duplicates() # There's a row for both teams, only need 1
    scheduleFrame = scheduleFrame.reset_index(drop=True)
            
    return scheduleFrame


In [7]:
# Get Single Game aggregation columns

def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        gameStats = cumestatsteam.CumeStatsTeam(game_ids=gameID,league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    data.at[1,'NICKNAME'] = awayTeamNickname
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [8]:
def getGameLogs(gameLogs, scheduleFrame):
    
    # Functions to prepare additional columns after gameLogs table loads
    def getHomeAwayFlag(gameDF):
        gameDF['HOME_FLAG'] = np.where((gameDF['W_HOME'] == 1) | (gameDF['L_HOME'] == 1), 1, 0)
        gameDF['AWAY_FLAG'] = np.where((gameDF['W_ROAD'] == 1) | (gameDF['L_ROAD'] == 1), 1, 0)

    def getTotalWinPctg(gameDF):
        gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID', 'SEASON'])['GAME_DATE'].rank(ascending=True)
        gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])['W'].cumsum()
        gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS'] / gameDF['TOTAL_GAMES_PLAYED']
        return gameDF.drop(['TOTAL_GAMES_PLAYED', 'TOTAL_WINS'], axis=1)

    def getHomeWinPctg(gameDF):
        gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])[
            'HOME_FLAG'].cumsum()
        gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])['W_HOME'].cumsum()
        gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS'] / gameDF['HOME_GAMES_PLAYED']
        return gameDF.drop(['HOME_GAMES_PLAYED', 'HOME_WINS'], axis=1)

    def getAwayWinPctg(gameDF):
        gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])[
            'AWAY_FLAG'].cumsum()
        gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])['W_ROAD'].cumsum()
        gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS'] / gameDF['AWAY_GAMES_PLAYED']
        return gameDF.drop(['AWAY_GAMES_PLAYED', 'AWAY_WINS'], axis=1)

    def getRollingOE(gameDF):
        gameDF['ROLLING_OE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])[
            'OFFENSIVE_EFFICIENCY'].transform(lambda x: x.rolling(3, 1).mean())

    def getRollingScoringMargin(gameDF):
        gameDF['ROLLING_SCORING_MARGIN'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])[
            'SCORING_MARGIN'].transform(lambda x: x.rolling(3, 1).mean())

    def getRestDays(gameDF):
        gameDF['LAST_GAME_DATE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID', 'SEASON'])[
            'GAME_DATE'].shift(1)
        gameDF['NUM_REST_DAYS'] = (gameDF['GAME_DATE'] - gameDF['LAST_GAME_DATE']) / np.timedelta64(1, 'D')
        return gameDF.drop('LAST_GAME_DATE', axis=1)

    start = time.perf_counter_ns()

    i = int(len(gameLogs) / 2)  # Can use a previously completed gameLog dataset

    while i < len(scheduleFrame):

        time.sleep(1)
        gameLogs = pd.concat([gameLogs, getSingleGameMetrics(scheduleFrame.at[i, 'GAME_ID'],
                                                             scheduleFrame.at[i, 'HOME_TEAM_ID'],
                                                             scheduleFrame.at[i, 'AWAY_TEAM_ID'],
                                                             scheduleFrame.at[i, 'AWAY_TEAM_NICKNAME'],
                                                             scheduleFrame.at[i, 'SEASON'],
                                                             scheduleFrame.at[i, 'GAME_DATE'])], ignore_index=True)

        end = time.perf_counter_ns()

        # Output time it took to load x amount of records
        if i % 100 == 0:
            mins = ((end - start) / 1e9) / 60
            print(i, str(mins) + ' minutes')

        i += 1

    # Get Table Level Aggregation Columns
    getHomeAwayFlag(gameLogs)
    gameLogs = getHomeWinPctg(gameLogs)
    gameLogs = getAwayWinPctg(gameLogs)
    gameLogs = getTotalWinPctg(gameLogs)
    getRollingScoringMargin(gameLogs)
    getRollingOE(gameLogs)
    gameLogs = getRestDays(gameLogs)

    return gameLogs.reset_index(drop=True)

In [9]:
#Get ScheduleFrame

seasons = [2020,2021,2022]
seasonType = 'Regular Season'

start = time.perf_counter_ns() # Track cell's runtime
scheduleFrame = getSeasonScheduleFrame(seasons,seasonType)
end = time.perf_counter_ns()

secs = (end-start)/1e9
mins = secs/60
print(mins)

1.9165125881833334


In [10]:
#Example Output of Single Game Metrics
getSingleGameMetrics(scheduleFrame.at[104,'GAME_ID'],scheduleFrame.at[104,'HOME_TEAM_ID'],
                     scheduleFrame.at[104,'AWAY_TEAM_ID'],scheduleFrame.at[104,'AWAY_TEAM_NICKNAME'],
                     scheduleFrame.at[104,'SEASON'],scheduleFrame.at[104,'GAME_DATE'])

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,BLK,PTS,AVG_REB,AVG_PTS,DQ,OFFENSIVE_EFFICIENCY,SCORING_MARGIN,SEASON,GAME_DATE,GAME_ID
0,Cleveland,Cavaliers,1610612739,1,0,1,0,0,0,2,...,3,117,49.0,117.0,0,0.588785,7.0,2020-21,2021-03-17,0022000620
1,OPPONENTS,Celtics,1610612738,0,1,0,0,0,1,0,...,9,110,48.0,110.0,0,0.567308,-7.0,2020-21,2021-03-17,0022000620


In [13]:
#Create the gameLogs DataFrame
gameLogs = pd.DataFrame()
gameLogs = getGameLogs(gameLogs,scheduleFrame)
gameLogs.to_csv('gameLogs.csv')

0 0.02152794445 minutes
100 2.3007218854166664 minutes
200 4.582067341666667 minutes
300 6.77925635765 minutes
400 9.0300534625 minutes
500 11.176855205566666 minutes
600 13.620991791666667 minutes
700 16.086919395149998 minutes
800 18.54540619375 minutes
900 21.55309391875 minutes
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
1000 25.321353897233333 minutes
1100 28.050189961816667 minutes
1200 30.682680011116666 minutes
1300 33.33927770625 minutes
1400 36.24090009028334 minutes
1500 38.93428164236666 minutes
1600 41.59157661598333 minutes
1700 44.28765557778333 minutes
1800 46.92095795278333 minutes
1900 49.655790135416666 minutes
2000 52.35570049306667 minutes
2100 55.19670161875 minutes
2200 57.97269561458334 minutes
2300 60.71288547778333 minutes
2400 63.552734988900006 minutes
2500 66.47141315208333 minutes
2600 69.24789167708333 minutes
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
2700 73.029771517

In [15]:
#Example Output of Game Logs
gameLogs[(gameLogs['TEAM_ID'] == 1610612737 ) & (gameLogs['SEASON'] == '2022-23')].sort_values('GAME_DATE')

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,GAME_DATE,GAME_ID,HOME_FLAG,AWAY_FLAG,HOME_WIN_PCTG,AWAY_WIN_PCTG,TOTAL_WIN_PCTG,ROLLING_SCORING_MARGIN,ROLLING_OE,NUM_REST_DAYS
4782,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2022-10-19,0022200005,1,0,1.000000,NaN,1.000000,10.000000,0.600000,NaN
4780,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,2022-10-21,0022200020,1,0,1.000000,NaN,1.000000,10.000000,0.568293,2.0
4778,Atlanta,Hawks,1610612737,0,1,0,1,0,0,0,...,2022-10-23,0022200038,1,0,0.666667,NaN,0.666667,1.000000,0.560148,2.0
4777,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,0,...,2022-10-26,0022200057,0,1,0.666667,1.000000,0.750000,-0.666667,0.554351,3.0
4775,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,1,...,2022-10-28,0022200070,0,1,0.666667,1.000000,0.800000,4.000000,0.604823,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4628,Atlanta,Hawks,1610612737,1,0,1,0,0,0,2,...,2023-04-02,0022201167,1,0,0.589744,0.410256,0.500000,-4.333333,0.581925,2.0
4627,OPPONENTS,Hawks,1610612737,1,0,0,0,1,0,1,...,2023-04-04,0022201182,0,1,0.589744,0.425000,0.506329,1.000000,0.592806,2.0
4624,Atlanta,Hawks,1610612737,1,0,1,0,0,0,1,...,2023-04-05,0022201191,1,0,0.600000,0.425000,0.512500,12.666667,0.607685,1.0
4622,Atlanta,Hawks,1610612737,0,1,0,1,0,0,1,...,2023-04-07,0022201205,1,0,0.585366,0.425000,0.506173,10.333333,0.602885,2.0


## Feature Engineered Dataset 
This function produces the dataset with all current feature variables for modeling and additional columns to perform a training set/validation set split. 

In [16]:
def getGameLogFeatureSet(gameDF):

    def shiftGameLogRecords(gameDF):
        gameDF['LAST_GAME_OE'] = gameLogs.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].shift(1)
        gameDF['LAST_GAME_HOME_WIN_PCTG'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_WIN_PCTG'].shift(1)
        gameDF['LAST_GAME_AWAY_WIN_PCTG'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_WIN_PCTG'].shift(1)
        gameDF['LAST_GAME_TOTAL_WIN_PCTG'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['TOTAL_WIN_PCTG'].shift(1)
        gameDF['LAST_GAME_ROLLING_SCORING_MARGIN'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_SCORING_MARGIN'].shift(1)
        gameDF['LAST_GAME_ROLLING_OE'] = gameDF.sort_values('GAME_DATE').groupby(['TEAM_ID','SEASON'])['ROLLING_OE'].shift(1)
    
    
    def getHomeTeamFrame(gameDF):
        homeTeamFrame = gameDF[gameDF['CITY'] != 'OPPONENTS']
        homeTeamFrame = homeTeamFrame[['LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_GAME_AWAY_WIN_PCTG','LAST_GAME_TOTAL_WIN_PCTG','LAST_GAME_ROLLING_SCORING_MARGIN','LAST_GAME_ROLLING_OE','W','TEAM_ID','GAME_ID','SEASON']]

        colRenameDict = {}
        for col in homeTeamFrame.columns:
            if (col != 'GAME_ID') & (col != 'SEASON') :
                colRenameDict[col] = 'HOME_' + col 

        homeTeamFrame.rename(columns=colRenameDict,inplace=True)

        return homeTeamFrame

    def getAwayTeamFrame(gameDF):
        awayTeamFrame = gameDF[gameDF['CITY'] == 'OPPONENTS']
        awayTeamFrame = awayTeamFrame[['LAST_GAME_OE','LAST_GAME_HOME_WIN_PCTG','NUM_REST_DAYS','LAST_GAME_AWAY_WIN_PCTG','LAST_GAME_TOTAL_WIN_PCTG','LAST_GAME_ROLLING_SCORING_MARGIN','LAST_GAME_ROLLING_OE','TEAM_ID','GAME_ID','SEASON']]

        colRenameDict = {}
        for col in awayTeamFrame.columns:
            if (col != 'GAME_ID') & (col != 'SEASON'):
                colRenameDict[col] = 'AWAY_' + col 

        awayTeamFrame.rename(columns=colRenameDict,inplace=True)

        return awayTeamFrame
    
    shiftGameLogRecords(gameLogs)
    awayTeamFrame = getAwayTeamFrame(gameLogs)
    homeTeamFrame = getHomeTeamFrame(gameLogs)
    
    return pd.merge(homeTeamFrame, awayTeamFrame, how="inner", on=[ "GAME_ID","SEASON"]).drop(['GAME_ID','AWAY_TEAM_ID','HOME_TEAM_ID'],axis=1)

In [17]:
modelData = getGameLogFeatureSet(gameLogs)

In [18]:
# Final Data Set before Train,Test, Validation Split
modelData

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,SEASON,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE
0,0.555556,0.685714,3.0,0.444444,0.563380,9.333333,0.571405,1,2020-21,0.612903,0.250000,2.0,0.228571,0.239437,-2.000000,0.605315
1,0.573913,0.676471,1.0,0.444444,0.557143,-0.666667,0.589472,1,2020-21,0.500000,0.305556,2.0,0.303030,0.304348,-18.000000,0.512228
2,0.584746,0.666667,2.0,0.444444,0.550725,8.666667,0.640592,1,2020-21,0.612403,0.500000,2.0,0.428571,0.463768,0.666667,0.567718
3,0.609756,0.656250,4.0,0.444444,0.544118,11.333333,0.653327,1,2020-21,0.539568,0.500000,2.0,0.441176,0.470588,0.666667,0.570866
4,0.512397,0.354839,1.0,0.558824,0.461538,11.000000,0.596498,1,2020-21,0.727273,0.656250,1.0,0.457143,0.552239,17.333333,0.633081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.531250,1.000000,2.0,0.500000,0.666667,-0.333333,0.547383,1,2022-23,0.533898,1.000000,3.0,0.000000,0.333333,-9.666667,0.549413
3536,0.626087,0.242424,2.0,0.212121,0.227273,-10.666667,0.564607,0,2022-23,0.605263,0.354839,2.0,0.277778,0.313433,-6.666667,0.534304
3537,0.517857,0.357143,2.0,0.264706,0.306452,7.666667,0.553343,1,2022-23,0.537815,0.266667,2.0,0.225806,0.245902,-8.000000,0.538467
3538,0.529412,0.240000,4.0,0.259259,0.250000,-0.333333,0.566698,1,2022-23,0.522523,0.304348,1.0,0.266667,0.283019,-6.666667,0.550896


In [19]:
modelData.to_csv('nbaHomeWinLossModelDataset.csv')